# PyTorch：神经网络模块nn

计算图和autograd是十分强大的工具，可以定义复杂的操作并自动求导；然而对于大规模的网络，autograd太过于底层。

在构建神经网络时，我们经常考虑将计算安排成**层**，其中一些具有**可学习的参数**，它们将在学习过程中进行优化。

TensorFlow里，有类似[Keras](https://github.com/fchollet/keras)，[TensorFlow-Slim](https://github.com/tensorflow/tensorflow/tree/master/tensorflow/contrib/slim)和[TFLearn](http://tflearn.org/)这种封装了底层计算图的高度抽象的接口，这使得构建网络十分方便。 

在PyTorch中，包`nn`完成了同样的功能。`nn`包中定义一组大致等价于层的**模块**。一个模块接受输入的tesnor，计算输出的tensor，而且还保存了一些内部状态比如需要学习的tensor的参数等。`nn`包中也定义了一组损失函数（loss functions），用来训练神经网络。 

这个例子中，我们用`nn`包实现两层的网络：

In [1]:
# 可运行代码见本文件夹中的 two_layer_net_nn.py
import torch

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# N是批大小；D是输入维度
# H是隐藏层维度；D_out是输出维度
N, D_in, H, D_out = 64, 1000, 100, 10

# 产生输入和输出随机张量
x = torch.randn(N, D_in, device=device)
y = torch.randn(N, D_out, device=device)


# 使用nn包将我们的模型定义为一系列的层。
# nn.Sequential是包含其他模块的模块，并按顺序应用这些模块来产生其输出。
# 每个线性模块使用线性函数从输入计算输出，并保存其内部的权重和偏差张量。
# 在构造模型之后，我们使用.to()方法将其移动到所需的设备。
model = torch.nn.Sequential(
            torch.nn.Linear(D_in, H),
            torch.nn.ReLU(),
            torch.nn.Linear(H, D_out),
        ).to(device)


# nn包还包含常用的损失函数的定义；
# 在这种情况下，我们将使用平均平方误差(MSE)作为我们的损失函数。
# 设置reduction='sum'，表示我们计算的是平方误差的“和”，而不是平均值;
# 这是为了与前面我们手工计算损失的例子保持一致，
# 但是在实践中，通过设置reduction='elementwise_mean'来使用均方误差作为损失更为常见。
loss_fn = torch.nn.MSELoss(reduction='sum')

learning_rate = 1e-4
for t in range(500):

    # 前向传播：通过向模型传入x计算预测的y。
    # 模块对象重载了__call__运算符，所以可以像函数那样调用它们。
    # 这么做相当于向模块传入了一个张量，然后它返回了一个输出张量。
    y_pred = model(x)
    
    # 计算并打印损失。我们传递包含y的预测值和真实值的张量，损失函数返回包含损失的张量。
    loss = loss_fn(y_pred, y)
    print(t, loss.item())
    
    # 反向传播之前清零梯度
    model.zero_grad()

    # 反向传播：计算模型的损失对所有可学习参数的导数（梯度）。
    # 在内部，每个模块的参数存储在requires_grad=True的张量中，
    # 因此这个调用将计算模型中所有可学习参数的梯度。
    loss.backward()

    # 使用梯度下降更新权重。
    # 每个参数都是张量，所以我们可以像我们以前那样可以得到它的数值和梯度
    with torch.no_grad():
        for param in model.parameters():
            param.data -= learning_rate * param.grad

0 589.2650756835938
1 544.6754150390625
2 505.7870788574219
3 471.72601318359375
4 441.301513671875
5 413.9375
6 389.0218505859375
7 366.3975830078125
8 345.60821533203125
9 326.4293212890625
10 308.5284118652344
11 291.85858154296875
12 276.22216796875
13 261.5279235839844
14 247.6606903076172
15 234.5232391357422
16 222.09951782226562
17 210.32373046875
18 199.12088012695312
19 188.49383544921875
20 178.3958740234375
21 168.78872680664062
22 159.64785766601562
23 150.9722442626953
24 142.72666931152344
25 134.9013671875
26 127.47538757324219
27 120.43806457519531
28 113.7598876953125
29 107.41643524169922
30 101.40911102294922
31 95.72021484375
32 90.34722137451172
33 85.25762939453125
34 80.44406127929688
35 75.89909362792969
36 71.5885238647461
37 67.51824951171875
38 63.66319274902344
39 60.03507995605469
40 56.62186813354492
41 53.39593505859375
42 50.35042953491211
43 47.486915588378906
44 44.79405975341797
45 42.25392150878906
46 39.87065124511719
47 37.629173278808594
48 35.51

410 6.032829696778208e-05
411 5.8611640270100906e-05
412 5.694355786545202e-05
413 5.532684372155927e-05
414 5.3756135457661e-05
415 5.223140760790557e-05
416 5.074905129731633e-05
417 4.931008879793808e-05
418 4.791326500708237e-05
419 4.65558550786227e-05
420 4.5239936298457906e-05
421 4.395968426251784e-05
422 4.2715393647085875e-05
423 4.150742825004272e-05
424 4.033457298646681e-05
425 3.919770824722946e-05
426 3.809118061326444e-05
427 3.70176239812281e-05
428 3.597621980588883e-05
429 3.4962748031830415e-05
430 3.3979951695073396e-05
431 3.3025477023329586e-05
432 3.209626447642222e-05
433 3.119434404652566e-05
434 3.031898449989967e-05
435 2.9467932108673267e-05
436 2.864056841644924e-05
437 2.7837490051751956e-05
438 2.7058515115641057e-05
439 2.6300187528249808e-05
440 2.5563938834238797e-05
441 2.484721517248545e-05
442 2.415318158455193e-05
443 2.3476990463677794e-05
444 2.2822221581009217e-05
445 2.218485860794317e-05
446 2.1566036593867466e-05
447 2.0964427676517516e-05
4